# First hands on with Sagemaker

With this project, we will learn how to use Amazon SageMaker to build, train, and deploy a machine learning (ML) model using the XGBoost ML algorithm. Task for this project is to develop a machine learning model to predict whether a customer will enroll for a certificate of deposit (CD). 
The model will be trained on the Bank Marketing Data Set that contains information on customer demographics, responses to marketing events, and external factors. A version of this dataset is publicly available from the Machine Learning Repository curated by the University of California, Irvine.

## 1. Import the required libraries and define the environment variables needed to prepare the data, train the ML model, and deploy the ML model.

In [2]:
bucket_name = 'mltestproject27091995'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


## 2. Create the S3 bucket to store data

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


## 3. Download the data to SageMaker instance and load the data into a dataframe.

In [3]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


## 4. Shuffle and split the data into training data and test data

I have considered training data (70% of customers) for model training loop. 
The test data (remaining 30% of customers) is used to evaluate the performance of the model on unseen data.

In [4]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


## 5. Train the ML model

In [5]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

We are going to use  Amazon SageMaker pre-built XGBoost algorithm.

In [6]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

We need to different hyper parameters.

In [7]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

We will train the model using gradient optimization on a ml.m4.xlarge instance.

In [8]:
xgb.fit({'train': s3_input_train})

2021-07-05 12:51:46 Starting - Starting the training job...
2021-07-05 12:52:09 Starting - Launching requested ML instancesProfilerReport-1625489505: InProgress
...
2021-07-05 12:52:43 Starting - Preparing the instances for training............
2021-07-05 12:54:47 Downloading - Downloading input data
2021-07-05 12:54:47 Training - Downloading the training image...
2021-07-05 12:55:14 Uploading - Uploading generated training modelArguments: train
[2021-07-05:12:55:09:INFO] Running standalone xgboost training.
[2021-07-05:12:55:09:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-07-05:12:55:09:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8418.82mb
[2021-07-05:12:55:09:INFO] Determined delimiter of CSV input is ','
[12:55:09] S3DistributionType set as FullyReplicated
[12:55:09] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[12:55:09] src/tree/updater_prune.cc:74:

## 6. Deploy the model

We need to deploy the model on a server and creates a SageMaker endpoint that can accessed. This step may take a few minutes to complete.

In [9]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-----------------------------------------------------!

 We need to predict whether customers in the test data enrolled for the bank product or not

In [11]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


## 7. Evaluate model performance

We are comparing the actual vs. predicted values in a table called a confusion matrix.

In [12]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



Based on the prediction, we can conclude that we have predicted a customer will enroll for a certificate of deposit accurately for 90% of customers in the test data, with a precision of 65% (278/429) for enrolled and 90% (10,785/11,928) for didn’t enroll.

## 8. Clean up

Delete the endpoint

In [13]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

Delete training artifacts and S3 bucket

In [14]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '9Z0Z35HB9TJDDH4F',
   'HostId': 'Hnjt1py1woxO+3PrFmPccrSdDTTTdx+li+hQMcjK8Mz9m8r6NWBOfGPKGoi9LdvQPXFbFIUB7tM=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Hnjt1py1woxO+3PrFmPccrSdDTTTdx+li+hQMcjK8Mz9m8r6NWBOfGPKGoi9LdvQPXFbFIUB7tM=',
    'x-amz-request-id': '9Z0Z35HB9TJDDH4F',
    'date': 'Mon, 05 Jul 2021 14:12:58 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-05-12-51-45-716/rule-output/ProfilerReport-1625489505/profiler-output/profiler-reports/LowGPUUtilization.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-05-12-51-45-716/rule-output/ProfilerReport-1625489505/profiler-output/profiler-reports/MaxInitializationTime.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/train/train.csv'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboo

### Remember to stop and delete SageMaker Notebook.